In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 데이터셋 로드
food = pd.read_csv('./foodData4_24.csv')

# 결측값 처리(Keywords열의 데이터 중 nan값 공백으로 처리)
# 현재 food 데이터에 일부 깨진데이터(?)가 있어서 결측값 처리해야 정상작동
food['Keywords'] = food['Keywords'].fillna('')

# TF-IDF(Term Frequency-Inverse Document Frequency)
# 텍스트 데이터의 통계적인 가중치를 계산, 처리해주는 기능(자연어 처리)

# TfidfVectorizer : 문서를 벡터 표현으로 바꿔주는 기능
# stop_words='english' => 영어의 일반적인 불용어 삭제(and, is, the, this 등 단어와 상관없는 문자)
tfidf = TfidfVectorizer(stop_words='english')

# food의 Keywords를 단어로 구별
# 단어로 구별된 Keywords를 TF-IDF 행렬로 생성
tfidf_matrix = tfidf.fit_transform(food['Keywords'])

# 사용자 키워드
# 각 사용자의 키워드를 딕셔너리로 조회
# 아래는 예시 데이터
users = {
    'user1': ['Dessert', 'Pork', 'Grains', 'Oven', 'Christmas', 'Brunch'],
    'user2': ['Dessert', 'Berries', 'Fruit', 'Low Protein', 'Brunch'],
    'user3': ['Egg Free', 'Fruit', 'Oven', 'Brunch', 'Berries']
}

# 추천 함수 정의
def get_recommendations(user_keywords, tfidf_matrix, food_data):
    # 사용자 키워드를 조회하여 공백(' ')에 조인하여 하나의 문자열로 처리
    # 사용자 키워드에 대한 TF-IDF 행렬 생성
    user_tfidf_matrix = tfidf.transform([' '.join(user_keywords)])
    
    # 사용자 키워드 행렬과 food 데이터의 키워드 행렬의 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
   
    # 음식 추천 알고리즘
    # 코사인 유사도 행렬에서 첫 번째 행(사용자 입력과 음식 간의 유사도)을 가져와서
    # 각 음식과의 유사도를 인덱스와 함께 리스트로 변환 => 각 키워드가 가진 유사도를 음식과 전체 조회하여 반환
    sim_scores = list(enumerate(cosine_sim[0]))
   
    # 유사도를 기준으로 리스트를 내림차순으로 정렬
    # key=lambda x: x[1]는 리스트의 각 요소를 유사도에 대해 정렬하기 위한 키 함수
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사도가 가장 높은 음식(sim_scores[0])을 제외하고 10개 반환
    # 가장 높은 음식을 반환하는 이유는 검색기록이 1개인 경우 그 데이터가 가장 높은 유사도를 가지게 되므로
    sim_scores = sim_scores[1:11]

    # sim_scores에 저장된 음식들의 인덱스를 저장
    # sim_scores에 저장된 형태 => (인덱스 번호, 유사도 점수)
    food_indices = [idx[0] for idx in sim_scores]

    # sim_scores에 저장된 음식에 대하여 iloc(food_indices)를 통하여 이름 조회
    return food_data.iloc[food_indices]['Name']

# 각 사용자에 대한 음식 추천 실행
for user, keywords in users.items():
    print(f"{user}의 유사한 음식 추천:")
    recommendations = get_recommendations(keywords, tfidf_matrix, food)
    print(recommendations)
    print()

user1의 유사한 음식 추천:
142321                        Coffee Chocolate Waffles
66308     Tina's Overnight Sausage &amp; Egg Casserole
15217                                          "Tomato
6498                      Very Soft Rolled Oat Cookies
14133                     Peggy's Foolproof  Pie Crust
111060                 Whole Wheat Banana Flax Muffins
28642                          Kiss My Grits Casserole
18910                     Sweet Cornmeal Corn Toasties
31738                            Croissants a L'orange
50218                         Italian Shepherd's Pie 2
Name: Name, dtype: object

user2의 유사한 음식 추천:
24478     Barefoot Contessa's Chocolate Dipped Strawberries
57229                              Cherry-Cream Crumble Pie
84104                                  Raspberry Cream Cake
125343                               The Best Blueberry Pie
153438                          Rustic Nectarine-Berry Tart
72238                           Double Blueberry Cookie Pie
103994                     